## Azure AI Search
### Using an AI Search Index for context data

In [3]:
# Set up the query for generating responses
from azure.identity import DefaultAzureCredential
from azure.core.credentials import AzureKeyCredential

from azure.identity import get_bearer_token_provider
from azure.search.documents import SearchClient
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

credential = AzureKeyCredential(os.getenv("AZURE_SEARCH_KEY"))
# token_provider = get_bearer_token_provider(credential, "https://cognitiveservices.azure.com/.default")

openai_client = AzureOpenAI(
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY")
)

search_client = SearchClient(
    endpoint=os.getenv("AZURE_SEARCH_ENDPOINT"),
    index_name=os.getenv("AZURE_SEARCH_INDEX_NAME"),
    credential=credential
)

result = search_client.get_document_count()

# This prompt provides instructions to the model
GROUNDED_PROMPT="""
You are a very smart AI researcher helping users find answers to their questions. Your answers are short and simple.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the chat model
query="Cache Fusion"

# Search results are created by the search client
# Search results are composed of the top 5 results and the fields selected from the search index
# Search results include the top 5 matches to your query
search_results = search_client.search(
    search_text=query,
    top=5,
    select="title, chunk",
    vector_queries=[{
        "kind": "text",
        "text": query,
        "fields": "text_vector",
        "k": 3,
    }],
)

sources_formatted = "\n\n".join([f'{document["title"]}:{document["chunk"]}' for document in search_results])

print(sources_formatted)

2405.16444v2.pdf:the KV caches
such that: If KV cache is not inside the system and is recom-
puted by the LLM engine in the runtime, we will move the
KV cache into CPU by torch.cpu() and open a thread to
write it back to disk in the background with torch.save().
During fetch_kv, we go through through the hash tables to
fetch KV cache for the fusor. The hash tables are kept in CPU
for their relatively small size (16MB for one million chunks).

7 Evaluation
Our key takeaways from the evaluation are:

• TTFT reduction: Compared to full KV recompute,
CacheBlend reduces TTFT by 2.2-3.3× over several mod-
els and tasks.

• High quality:Compared with full KV reuse,CacheBlend
improves quality from 0.15 to 0.35 in F1-score and Rouge-L
score, while having no more than 0.01-0.03 quality drop
compared to full KV recompute and prefix caching.

• Higher throughput: At the same TTFT, CacheBlend
can increase throughput by up to 5× compared with full
KV recompute and 3.3× compared with prefix caching.


### Simple Q&A Test (no evaluation)

In [4]:
# Send the search results and the query to the LLM to generate a response based on the prompt.
response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT")
)

# Here is the response from the chat model.
print(response.choices[0].message.content)

- **Cache Fusion in CacheBlend**:
  - CacheBlend combines precomputed Key-Value (KV) caches from multiple text chunks to speed up large language model (LLM) inferences without fully recomputing KV caches. 
  - The KV cache fusor integrates precomputed KV caches while selectively recomputing KV values for partial updates.
  - This approach reduces time-to-first-token (TTFT) by 2.2–3.3× and increases throughput by up to 5× compared to full KV recompute.

- **KV Cache Handling**:
  - KV caches are stored in slower storage devices (e.g., NVMe SSDs) and are brought into memory when needed.
  - During LLM input processing, the fusor recomputes necessary portions of the KV cache while loading precomputed segments, maintaining efficiency.

- **System Workflow**:
  - A loading controller queries a KV cache manager to determine the storage location of KV caches and computes the selective recompute ratio.
  - KV caches are placed in GPU memory in a queue for computation and fusion.
  - After KV c

### Setup Evaluation Logic

In [5]:
from typing import List, Dict, Any, Optional
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

def call_to_your_ai_application(query: str) -> str:
    # logic to call your application
    # use a try except block to catch any errors

    token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
    deployment = os.getenv("AZURE_OPENAI_DEPLOYMENT")
    endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
    client = AzureOpenAI(
        azure_endpoint=endpoint,
        api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
        api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        # azure_ad_token_provider=token_provider,
    )
    completion = client.chat.completions.create(
        model=deployment,
        messages=[
            {
                "role": "user",
                "content": query,
            }
        ],
        max_tokens=800,
        temperature=0.7,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None,
        stream=False,
    )
    
    message = completion.to_dict()["choices"][0]["message"]
    return message["content"]


async def callback(
    messages: List[Dict],
    stream: bool = False,
    session_state: Any = None,  # noqa: ANN401
    context: Optional[Dict[str, Any]] = None,
) -> dict:
    messages_list = messages["messages"]
    # get last message
    latest_message = messages_list[-1]
    query = latest_message["content"]
    context = None
    # call your endpoint or ai application here
    response = call_to_your_ai_application(query)
    # we are formatting the response to follow the openAI chat protocol format
    formatted_response = {
        "content": response,
        "role": "assistant",
        "context": {
            "citations": None,
        },
    }
    messages["messages"].append(formatted_response)
    return {"messages": messages["messages"], "stream": stream, "session_state": session_state, "context": context}

### Run Simulator to synthesize Q&A pairs from search query and index context

In [7]:
from azure.ai.evaluation import AzureOpenAIModelConfiguration

judge_model = AzureOpenAIModelConfiguration(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    azure_deployment="gpt-4o",
    api_version=os.getenv("AZURE_OPENAI_API_VERSION")
)


In [8]:
from azure.ai.evaluation.simulator import Simulator

simulator = Simulator(model_config=judge_model)

outputs = await simulator(
    target=callback,
    text=sources_formatted,
    num_queries=4,
    max_conversation_turns=1,
    tasks=[
        f"I want to learn more about {query}",
    ],
)

Class Simulator: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
c:\Users\charendt\code\evaluations-getting-started\.venv\Lib\site-packages\azure\ai\evaluation\simulator\_simulator.py:151: UserWarning: You have specified 'num_queries' > len('tasks') (4 > 1). All tasks will be used for generation and the remaining 3 lines will be simulated in task-free mode
  warnings.warn(
Generating: 100%|████████████████████████████████████████████████| 4/4 [00:20<00:00,  5.02s/message]


In [9]:
from pathlib import Path
import json

output_file = Path("output.json")
with output_file.open("a") as f:
    json.dump(outputs, f)

### Running evaluations on the simulated data
Here we will try to run GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator on the output data from the simulator.

From the documentation we know that running those evaluators need the following data: `query`, `response`, `context`, `ground_truth`

For simplicity's sake, we can use our source document `text` as both `context` and `ground_truth`. This step only evaluates the first user message and first response from your AI Application for each of the simulated conversations.

In [10]:
eval_input_data_json_lines = ""
for output in outputs:
    query = None
    response = None
    context = sources_formatted
    ground_truth = sources_formatted
    for message in output["messages"]:
        if message["role"] == "user":
            query = message["content"]
        if message["role"] == "assistant":
            response = message["content"]
    if query and response:
        eval_input_data_json_lines += (
            json.dumps(
                {
                    "query": query,
                    "response": response,
                    "context": context,
                    "ground_truth": ground_truth,
                }
            )
            + "\n"
        )

Store the output in a file

In [11]:
eval_input_data_file = Path("eval_input_data.jsonl")

with eval_input_data_file.open("w") as f:
    f.write(eval_input_data_json_lines)

### Run evaluation
`QAEvaluator` is a composite evaluator which runs GroundednessEvaluator, RelevanceEvaluator, CoherenceEvaluator, FluencyEvaluator, SimilarityEvaluator, F1ScoreEvaluator

Optionally set the azure_ai_project to upload the evaluation results to Azure AI Studio.

In [12]:
azure_ai_project = {
    "subscription_id": os.getenv("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.getenv("AZURE_RESOURCE_GROUP"),
    "project_name": os.getenv("AZURE_PROJECT_NAME"),
}

In [13]:
from azure.ai.evaluation import evaluate, QAEvaluator

qa_evaluator = QAEvaluator(model_config=judge_model)

eval_output = evaluate(
    data=str(eval_input_data_file),
    evaluators={"QAEvaluator": qa_evaluator},
    evaluator_config={
        "QAEvaluator": {
            "column_mapping": {
                "query": "${data.query}",
                "response": "${data.response}",
                "context": "${data.context}",
                "ground_truth": "${data.ground_truth}",
            }
        }
    },
    azure_ai_project=azure_ai_project,  # optional to store the evaluation results in Azure AI Studio
    output_path="./myevalresults.json",  # optional to store the evaluation results in a file
)

[2025-02-24 18:26:50 +0100][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2025-02-24 18:26:50 +0100][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ie0kadbq_20250224_182650_474617, log path: C:\Users\charendt\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ie0kadbq_20250224_182650_474617\logs.txt


2025-02-24 18:26:50 +0100   23152 execution.bulk     INFO     Current thread is not main thread, skip signal handler registration in BatchEngine.
2025-02-24 18:27:13 +0100   23152 execution.bulk     INFO     Finished 4 / 4 lines.
2025-02-24 18:27:13 +0100   23152 execution.bulk     INFO     Average execution time for completed lines: 5.58 seconds. Estimated time for incomplete lines: 0.0 seconds.
======= Run Summary =======

Run name: "azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ie0kadbq_20250224_182650_474617"
Run status: "Completed"
Start time: "2025-02-24 18:26:50.472621+01:00"
Duration: "0:00:22.852234"
Output path: "C:\Users\charendt\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ie0kadbq_20250224_182650_474617"

======= Combined Run Summary (Per Evaluator) =======

{
    "QAEvaluator": {
        "status": "Completed",
        "duration": "0:00:22.852234",
        "completed_lines": 4,
        "failed_lines": 0,
        